In [1]:
import findspark
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark import SparkConf

from inno_syncer.main import start_job
from inno_syncer.sync.config import get_config

this_dir = Path().cwd()


findspark.init()

conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.4.3,org.postgresql:postgresql:42.7.2",
)
conf.set("spark.sql.sources.default", "iceberg")
conf.set("spark.sql.execution.pyarrow.enabled", "true")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.spark_catalog.type", "hive")
conf.set("hive.metastore.uris", "thrift://localhost:9083")
conf.set(
    "spark.sql.extensions",
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
)
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.sql("USE spark_catalog;")


24/03/30 14:50:10 WARN Utils: Your hostname, manhtct resolves to a loopback address: 127.0.1.1; using 172.31.155.66 instead (on interface eth0)
24/03/30 14:50:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/manhtct/innoorient/inno_syncer/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/manhtct/.ivy2/cache
The jars for the packages stored in: /home/manhtct/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b7e8dc8b-737c-4e77-8bcf-cd9ce563e7df;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.3 in central
	found org.postgresql#postgresql;42.7.2 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 125ms :: artifacts dl 7ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.3 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | numb

24/03/30 14:50:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


DataFrame[]

In [3]:
spark.sql("SHOW DATABASES").show()

+----------+
| namespace|
+----------+
|   default|
|github_raw|
| mysql_raw|
+----------+



In [5]:
spark.sql("SHOW TABLES IN spark_catalog.github_raw;").show()

+----------+----------+-----------+
| namespace| tableName|isTemporary|
+----------+----------+-----------+
|github_raw|    commit|      false|
|github_raw|repository|      false|
+----------+----------+-----------+



In [16]:
spark.sql("SELECT * FROM spark_catalog.github_raw.commit LIMIT 10").show()

+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+
|                 sha|repository_id|     committer_date|        author_email|         author_name|     committer_email|     committer_name|             message|        author_date|    _fivetran_synced|
+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+
|4647bf23ac656195d...|    605490786|2023-02-23 16:15:02|19133035@student....| Trần Công Tuấn Mạnh|19133035@student....|Trần Công Tuấn Mạnh|        first commit|2023-02-23 16:15:02|2024-02-28 09:39:...|
|0ef98692d584a0884...|    730033690|2022-07-12 03:15:14|koli.mikael@gmail...|         Mikael Koli|koli.mikael@gmail...|        Mikael Koli|  rem: Pandas import|2022-07-12 03:15:14|2024-02-28 1

In [8]:
spark.sql("DESC DATABASE EXTENDED spark_catalog.github_raw;").show()

+--------------+--------------------+
|     info_name|          info_value|
+--------------+--------------------+
|Namespace Name|          github_raw|
|       Comment|                    |
|      Location|file:/home/manhtc...|
|         Owner|             manhtct|
|    Properties|                    |
+--------------+--------------------+



In [12]:
spark.sql("DESC TABLE EXTENDED spark_catalog.github_raw.commit;").toPandas()

,col_name,data_type,comment
0,sha,string,
1,repository_id,bigint,
2,committer_date,timestamp,
3,author_email,string,
4,author_name,string,
5,committer_email,string,
6,committer_name,string,
7,message,string,
8,author_date,timestamp,
9,_fivetran_synced,timestamp,


In [14]:
spark.sql("SELECT * FROM spark_catalog.github_raw.commit.history").show()

+--------------------+-------------------+---------+-------------------+
|     made_current_at|        snapshot_id|parent_id|is_current_ancestor|
+--------------------+-------------------+---------+-------------------+
|2024-03-30 14:37:...|6782685386683357026|     null|               true|
+--------------------+-------------------+---------+-------------------+

